In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/testcodegames/jsonformatter.csv
/kaggle/input/testcodegames/jsonformatter.xlsx
/kaggle/input/testcodegames/jsonformatter (1).xlsx


In [2]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW, get_linear_schedule_with_warmup

In [3]:
!pip install accelerate

In [3]:
import accelerate

print(accelerate.__version__)

0.29.3


In [4]:
df = pd.read_excel('/kaggle/input/testcodegames/jsonformatter (1).xlsx')

In [5]:
df['use_case_document']

0      roles\nrole Trader\ndescription Individuals or...
1      Use Cases for a Financial Trading Platform for...
2      Use Cases for a Financial Trading Platform\nAc...
3      Invoice Management System\nActors\nAdmin Users...
4      Invoice Management System\nUse Cases\nTitle Cr...
                             ...                        
232    Title: Inventory Management System\nDescriptio...
233    Title: Goods Receipt Process\nDescription:\nTh...
234    Title: Investment Request Process\nDescription...
235    Title: Accounts Payable Process\nDescription:\...
236    Title: Accounts Receivable Process\nDescriptio...
Name: use_case_document, Length: 237, dtype: object

In [6]:
import re
from nltk.corpus import stopwords

def preprocess_text_regex(text):
    """
    This function removes special characters, numbers, and punctuation from a string using regular expressions,
    and removes stop words.

    Args:
        text: The string to preprocess.

    Returns:
        A new string with only letters and spaces, with stop words removed.
    """
    # Regex to match anything except letters and spaces
    pattern = r"[^a-zA-Z\s]"
    
    # Split the text into lines
    lines = text.split('\n')
    
    # Preprocessed text
    cleaned_text = ''
    
    # Process each line
    for line in lines:
        # Remove special characters and numbers
        cleaned_line = re.sub(pattern, "", line)
        
        # Remove stop words
        filtered_words = [word for word in cleaned_line.split() if word.lower() not in stopwords.words('english')]
        
        # Join the filtered words and add to cleaned_text
        cleaned_text += ' '.join(filtered_words) + ' '
    
    # Return the final cleaned text, stripping any extraneous whitespace
    return cleaned_text.strip()


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [8]:
for i in range(0,len(df['use_case_document'])):
    if df['use_case_document'] is not 'NAN':
      df['use_case_document'][i] = preprocess_text_regex(str(df['use_case_document'][i]))

<>:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_34/2682546286.py:2: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if df['use_case_document'] is not 'NAN':


In [9]:
print(df['use_case_document'][193])

Use Case Document Contract Management System Construction Firm  Use Case ID UC Use Case Name Create Manage Construction Contract  Actors Primary Actor Contracts Manager Secondary Actors Project Manager Subcontractor  Preconditions  Project information available system Necessary approvals place eg budget Postconditions  signed contract document exists system relevant stakeholders access contract Contract milestones deliverables clearly defined Main Flow  Contracts Manager initiates contract creation project System prompts user input project details select contract template Contracts Manager enters details like scope work price timelines System validates entries ensures consistency project information Contracts Manager assigns relevant stakeholders Project Manager Subcontractor System notifies assigned stakeholders allows review contract Stakeholders review provide feedback contract System tracks revisions maintains audit trail Contracts Manager finalizes contract initiates esigning work

In [10]:
# Define constants
source_max_len = 512
target_max_len = 512
batch_size = 2
epochs = 8

In [11]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

In [12]:
# Initialize tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained('Mithil/gpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model = GPT2LMHeadModel.from_pretrained('Mithil/gpt2').to('cuda')

tokenizer_config.json:   0%|          | 0.00/696 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/907 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [13]:
!pip install datasets

In [13]:
from datasets import Dataset
# import pandas as pd
# df = pd.DataFrame({"a": [1, 2, 3]})
dataset = Dataset.from_pandas(df)

In [14]:
def full_article(example):
  return {'Generate Use Case Document': f"Prompt:{example['prompt']}\n\nUse_Case_Document:{example['use_case_document']}"}

In [15]:
# Define optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

In [16]:
dataset = dataset.map(full_article)

Map:   0%|          | 0/237 [00:00<?, ? examples/s]

In [17]:
dataset

Dataset({
    features: ['prompt', 'use_case_document', 'Generate Use Case Document'],
    num_rows: 237
})

In [18]:
dataset['Generate Use Case Document'][148]

'Prompt:Customer Relationship Management Application Use Cases\n\nUse_Case_Document:Manage Customer Information  Description Allows users store update retrieve customer information contact details communication history preferences interactions Actors Sales Representatives Customer Support Staff Preconditions User must authenticated authorized Customer information must available system Postconditions Customer information updated accessible future interactions Steps User navigates customer management section User searches specific customer using filters search criteria User views edits customers details contact information preferences notes User saves changes updating customers information system Track Customer Interactions  Description Enables users track manage customer interactions across various channels including calls emails meetings support tickets Actors Sales Representatives Customer Support Staff Preconditions User must authenticated authorized Customer interactions must logged

In [19]:
def tokenize(example):
  outputs = tokenizer(example['Generate Use Case Document'],truncation=True,max_length=1024,return_overflowing_tokens=False)
  return outputs

In [20]:
tokenized_dataset = dataset.map(tokenize,batched=True)

Map:   0%|          | 0/237 [00:00<?, ? examples/s]

In [21]:
tokenized_dataset

Dataset({
    features: ['prompt', 'use_case_document', 'Generate Use Case Document', 'input_ids', 'attention_mask'],
    num_rows: 237
})

In [22]:
from transformers import AutoConfig

In [23]:
config =  AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_ctx=1024,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id
)

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [24]:
model = GPT2LMHeadModel(config)

In [25]:
model_size = sum(t.numel() for t in model.parameters())

In [35]:
print(f"GPT-2 size :{model_size/1000**2:.1f}M parameters")

GPT-2 size :124.4M parameters


In [36]:
from transformers import DataCollatorForLanguageModeling

2024-05-21 17:01:06.934736: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-21 17:01:06.934833: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-21 17:01:07.058145: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [37]:
tokenizer.pad_token = tokenizer.eos_token

In [38]:
data_collator = DataCollatorForLanguageModeling(tokenizer,mlm=False)

In [39]:
from huggingface_hub import notebook_login

In [40]:
notebook_login()

In [41]:
from transformers import Trainer,TrainingArguments

In [42]:
  training_args = TrainingArguments(
    output_dir='./code_games_gpt2',
    hub_model_id='gpt2',
    evaluation_strategy='epoch',
    auto_find_batch_size=True,
    num_train_epochs=20,
    gradient_accumulation_steps=8,
    weight_decay=0.1,
    lr_scheduler_type='cosine',
    learning_rate=1e-4,
    # fp16=True,
    push_to_hub=True,
    logging_steps=10
)

In [43]:
 trainer = Trainer(
     model=model,
     tokenizer=tokenizer,
     args=training_args,
     data_collator=data_collator,
     train_dataset=tokenized_dataset,
     eval_dataset=tokenized_dataset
 )

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [44]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
0,No log,8.281633
2,8.253000,5.983854
4,7.409800,5.179191
6,6.269800,4.628228
8,5.809200,4.266767
10,5.350100,4.002358
12,5.127800,3.810553
14,4.927900,3.709751
16,4.812900,3.660059
18,4.864300,3.653035


TrainOutput(global_step=140, training_loss=6.025015776497977, metrics={'train_runtime': 426.517, 'train_samples_per_second': 11.113, 'train_steps_per_second': 0.328, 'total_flos': 1076506841088000.0, 'train_loss': 6.025015776497977, 'epoch': 18.67})

In [45]:
trainer.push_to_hub()

events.out.tfevents.1716310898.04500254204e.34.0:   0%|          | 0.00/17.9k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Mithil/gpt2/commit/0fc0548975f951f00ce14d1153c373a5b2474eb1', commit_message='End of training', commit_description='', oid='0fc0548975f951f00ce14d1153c373a5b2474eb1', pr_url=None, pr_revision=None, pr_num=None)

In [26]:
import torch
from transformers import pipeline

2024-05-22 06:36:58.200060: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-22 06:36:58.200194: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-22 06:36:58.308364: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [27]:
pipe = pipeline("text-generation",model="Mithil/gpt2")

In [28]:
prompt = f"Generate Use Case Document:\n\nPrompt: {df['prompt'][185]}\n\nUse_Case_Document:"

In [29]:
prompt

'Generate Use Case Document:\n\nPrompt: Library Management System Use Case Document to return book\n\nUse_Case_Document:'

In [31]:
prompt = 'Generate Use Case Document:\n\nPrompt: User Registration Use Case Document\n\nUse_Case_Document:'

In [32]:
print(pipe(prompt,max_new_tokens=128)[0]['generated_text'])

Generate Use Case Document:

Prompt: User Registration Use Case Document

Use_Case_Document:Use Case Document Description use case document for Preconditions User inventory levels Actors User adds new Enables users plan User must logged Postconditions User creates new employee module User must available Postconditions User must logged Postconditions employee details review based updated details necessary including Postconditions User accesses identify information eg management selects data logs User adds new stock User system identify Steps User selects new employee details User must logged system handle Postconditions User selects details User selects existing invoice including data Postconditions User adds track User selects new employee Allows users manage review Postconditions User selects review contract User receives Postconditions User must logged User must
